In [ ]:
from dotenv import load_dotenv, find_dotenv

assert load_dotenv(find_dotenv(usecwd=False)), "The .env file was not loaded."

from concurrent.futures import ProcessPoolExecutor, as_completed
from pathlib import Path

import numpy as np
import pandas as pd
import torch
from skopt.space import Categorical, Integer, Real
import drn

from generate_synthetic_dataset import generate_synthetic_gamma_lognormal
from hyperparameter_tune import hyperparameter_tune
from hyperparameter_tuning_objectives import (
    objective_cann,
    objective_ddr,
    objective_drn,
    objective_mdn,
)

torch.set_num_threads(1)

In [ ]:
accelerator = "gpu" if torch.cuda.is_available() else "cpu"
print(f"Using {accelerator} for training.")

In [ ]:
PARALLEL = (accelerator == "cpu")
max_workers = 4 if PARALLEL else 1
executor = ProcessPoolExecutor(max_workers=max_workers)
futures = []

In [ ]:
MODEL_DIR = Path("models/synth")
MODEL_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
hp_opts = {
    "n_calls": 125,
    "n_initial_points": 25,
    "verbose": False,
}

In [ ]:
features, target, means, dispersion = generate_synthetic_gamma_lognormal(20000, seed=1)
df = pd.concat([features, target], axis=1)
df.to_csv("data/synth.csv", index=False)
df

In [ ]:
(
    x_train,
    x_val,
    x_test,
    y_train,
    y_val,
    y_test,
    x_train_raw,
    x_val_raw,
    x_test_raw,
    num_features,
    cat_features,
    all_categories,
    ct,
) = drn.split_and_preprocess(
    features, target, ["X_1", "X_2"], [], seed=42, num_standard=False
)
x_train

In [ ]:
X_train = torch.Tensor(x_train.values)
Y_train = torch.Tensor(y_train.values)
X_val = torch.Tensor(x_val.values)
Y_val = torch.Tensor(y_val.values)

In [ ]:
distribution = "gamma"  # distributional assumption for the GLM, CANN, MDN

In [ ]:
# Generate random seeds for the various training runs
np.random.seed(2025)
seeds = [int(s) for s in np.random.randint(0, 2**32 - 1, size=4)]
seeds

### GLM

In [ ]:
glm = drn.GLM.from_statsmodels(X_train, Y_train, distribution=distribution)
torch.save(glm, MODEL_DIR / "glm.pkl")

### CANN

In [ ]:
name = "cann"
path = MODEL_DIR / f"{name}_hp.pkl"

space_cann_synth = [
    Integer(1, 4, name="num_hidden_layers"),
    Categorical([16, 32, 64, 128, 256, 512], name="hidden_size"),
    Real(0.0, 0.5, name="dropout_rate"),
    Real(0.0002, 0.01 / 2, name="lr"),
    Categorical([128, 256, 512], name="batch_size"),
]

futures.append(executor.submit(
    hyperparameter_tune,
    objective_cann,
    space_cann_synth,
    path,
    seed=seeds[0],
    top_n=1,
    gp_minimize_opts=hp_opts,
    X_train=X_train,
    Y_train=Y_train,
    X_val=X_val,
    Y_val=Y_val,
    accelerator=accelerator,
    glm=glm,
    patience=10,
))

### MDN

In [ ]:
name = "mdn"
path = MODEL_DIR / f"{name}_hp.pkl"

space_mdn_synth = [
    Integer(1, 4, name="num_hidden_layers"),
    Categorical([16, 32, 64, 128, 256, 512], name="hidden_size"),
    Real(0.0, 0.5, name="dropout_rate"),
    Real(0.0002, 0.01 / 2, name="lr"),
    Integer(2, 10, name="num_components"),
    Categorical([128, 256, 512], name="batch_size"),
]

futures.append(executor.submit(
    hyperparameter_tune,
    objective_mdn,
    space_mdn_synth,
    path,
    seed=seeds[1],
    top_n=1,
    gp_minimize_opts=hp_opts,
    X_train=X_train,
    Y_train=Y_train,
    X_val=X_val,
    Y_val=Y_val,
    accelerator=accelerator,
    distribution="gamma",
    patience=10,
))

### DDR

In [ ]:
name = "ddr"
path = MODEL_DIR / f"{name}_hp.pkl"

space_ddr_synth = [
    Integer(1, 4, name="num_hidden_layers"),
    Categorical([16, 32, 64, 128, 256, 512], name="hidden_size"),
    Real(0.0, 0.5, name="dropout_rate"),
    Real(0.0001, 0.01 / 2, name="lr"),
    Categorical(np.linspace(0.01, 0.05, 9), name="proportion"),
    Categorical([128, 256, 512], name="batch_size"),
]

futures.append(executor.submit(
    hyperparameter_tune,
    objective_ddr,
    space_ddr_synth,
    path,
    seed=seeds[2],
    top_n=1,
    gp_minimize_opts=hp_opts,
    X_train=X_train,
    Y_train=Y_train,
    X_val=X_val,
    Y_val=Y_val,
    accelerator=accelerator,
    patience=10,
))


### DRN

In [ ]:
name = "drn"
path = MODEL_DIR / f"{name}_hp.pkl"

drn_defaults = dict(
    num_hidden_layers=3,
    hidden_size=256,
    dropout_rate=0.1,
    batch_size=256,
)

space_drn_synth = [
    Real(1e-4, 5e-3, name="lr", prior="log-uniform"),
    Real(5e-4, 5e-2, name="kl_alpha", prior="log-uniform"),
    Real(1e-4, 1e-1, name="mean_alpha", prior="log-uniform"),
    Real(1e-2, 1e-1, name="dv_alpha", prior="log-uniform"),
    Categorical([0.025, 0.05], name="proportion"),
    Categorical([5, 10], name="min_obs"),
]

futures.append(executor.submit(
    hyperparameter_tune,
    objective_drn,
    space_drn_synth,
    path,
    seed=seeds[3],
    top_n=1,
    gp_minimize_opts=hp_opts,
    X_train=X_train,
    Y_train=Y_train,
    X_val=X_val,
    Y_val=Y_val,
    accelerator=accelerator,
    glm=glm,
    criteria="CRPS",
    kl_direction="forwards",
    patience=10,
    **drn_defaults,
))

In [ ]:
# Collect results (blocks only if jobs still running)
results = []
for fut in as_completed(futures):
    res = fut.result()
    results.append(res)

executor.shutdown()

In [ ]:
for name, best_models in zip(["cann", "mdn", "ddr", "drn"], results):
    best_model = best_models[0]
    torch.save(best_model, MODEL_DIR / f"{name}.pkl")